In [81]:
from bs4 import BeautifulSoup
import requests
import matplotlib
import time
from datetime import timedelta, datetime
import tempfile
from gtts import gTTS
from pygame import mixer
mixer.init()

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


In [82]:
PTT_URL = 'https://www.ptt.cc'
def get_web_page(url):
    time.sleep(0.5)  # 每次爬取前暫停 0.5 秒以免被 PTT 網站判定為大量惡意爬取
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html.parser')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').string.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            if d.find('div', 'nrec').string:
                try:
                    push_count = int(d.find('div', 'nrec').string)  # 轉換字串為數字
                except ValueError:  # 若轉換失敗，不做任何事，push_count 保持為 0
                    pass

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').string
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count
                })
    return articles, prev_url

def checkformat(soup, class_tag, data, index, url):
    content = soup.select(class_tag)[index].text
    return content

In [89]:
def parse(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    date = checkformat(soup, '.article-meta-value', 'date', 3, dom)
    content = soup.find(id="main-content").text
    target_content = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
    #去除掉 target_content
    content = content.split(target_content)
    
    content = content[0].split(date)
    print(content)
    if content[1]:
        main_content = content[1].replace('--', '')
        #印出內文
        print(main_content)
        return main_content

def speak(sentence):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts = gTTS(text=sentence, lang='zh-tw')
        tts.save("{}.mp3".format(fp.name))
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play()

In [93]:
if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/joke/index.html')
    
    yesterday = datetime.today() + timedelta(-1)
    if current_page:
        articles = []  # 全部的今日文章
        date =yesterday.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, date)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, date)
    for article in articles:
        page = get_web_page(PTT_URL + article['href'])
    if page:
        content = parse(page)
    speak(content)
    


['作者ReiKuromiya (Rei Kuromiya)看板joke標題[ＸＤ] Gif圖時間', '\n\n01\nhttps://i.imgur.com/twwO7PP.gif\n\n\n02\nhttps://i.imgur.com/wNYPvPQ.gif\n\n\n03\nhttps://i.imgur.com/8ou2VXl.gif\n\n\n04\nhttps://i.imgur.com/vz3uKXu.gif\n\n\n06\nhttps://i.imgur.com/QK7TvIy.gif\n\n\n07\nhttps://i.imgur.com/KEzeaLz.gif\n\n\n08\nhttps://i.imgur.com/9vOpmtt.gif\n\n\n10\nhttps://i.imgur.com/4EAPC0R.gif\n\n\n11\nhttps://i.imgur.com/ByU2ZbF.gif\n\n\n12\nhttps://i.imgur.com/sPuchyK.gif\n\n\n13\nhttps://i.imgur.com/piP86U8.gif\n\n\n14\nhttps://i.imgur.com/NuOiNVV.gif\n\n\n15\nhttps://i.imgur.com/YjDxSku.gif\n\n\n16\nhttps://i.imgur.com/69x3beD.gif\n\n\n18\nhttps://i.imgur.com/gd0WwdQ.gif\n\n\n19\nhttps://i.imgur.com/KomQXhs.gif\n\n\n20\nhttps://i.imgur.com/PJHE6hO.gif\n\n\n21\nhttps://i.imgur.com/MmuPaWk.gif\n\n\n22\nhttps://i.imgur.com/ugx9OQP.gif\n\n\n23\nhttps://i.imgur.com/TOE6qLw.gif\n\n\n24\nhttps://i.imgur.com/Zwqbujb.gif\n\n\n26\nhttps://i.imgur.com/Q8pnmxx.gif\n\n\n\n--\n\n新來的天才同事\n\nhttps://i.imgur.com/sB